# Импорт необходимых библиотек

In [1]:
!pip install --proxy 10.0.0.1:3128 gensim emoji pandas nltk pymorphy2

Using legacy 'setup.py install' for emoji, since package 'wheel' is not installed.
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for emoji: started
    Running setup.py install for emoji: finished with status 'done'
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [55]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np
from random import randint

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [7]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

# Датасет ВК

## Загрузка датасета

In [8]:
colnames = ['comment',]

In [9]:
comments = pd.read_csv('datasets/vk.csv', names=colnames, header=0, sep='\t')

In [10]:
comments

,comment
0,['😭']
1,"['хранить', 'господь', 'неравнодушный', 'челов..."
2,"['евгений', 'еслиб', 'бог', 'такой', 'допускать']"
3,"['кирилл', 'полностью', 'согласный', '😢']"
4,"['кирилл', 'бог', 'путь', 'неисповедимый']"
...,...
50779,"['с', 'фамилия', '🙄']"
50780,"['наш', 'день', 'повезти', 'мужик', 'фамилия',..."
50781,"['чудо', 'чудной', 'диво', 'дивный']"
50782,"['пригреться', 'уснуть', 'свариться', 'лягушка..."


In [11]:
def str_to_list(i):
    new_list = eval(i)
    return new_list

In [12]:
comments['comment'] = comments['comment'].apply(str_to_list)

### Выделение эмодзи флагов

In [13]:
def split(s):
    return[char for char in s]

reg_ident = ['🇷', '🇺', '🇦', '🇸', '🇧', '🇨', '🇩', '🇪', '🇫', '🇬', '🇭', '🇮', '🇯', '🇰', '🇱', '🇲', '🇳', '🇴', '🇵', '🇶', '🇸', '🇹', 'v', '🇼', '🇽', '🇾', '🇿']

def get_flags(doc):
    for item in doc:
        new_item = []
        for c in split(item):
            if c in reg_ident:
                if len(new_item) < 2:
                    new_item.append(c)
        if len(new_item) == 2 and new_item[0] != new_item[1]:
            try:
                i = doc.index(item)
                doc[i] = new_item[0] + new_item[1]
            except ValueError:
                doc[i] = ''
    return doc

In [14]:
comments['comment'] = comments['comment'].apply(get_flags)

In [15]:
comments

,comment
0,[😭]
1,"[хранить, господь, неравнодушный, человек, 🙏, 🕊]"
2,"[евгений, еслиб, бог, такой, допускать]"
3,"[кирилл, полностью, согласный, 😢]"
4,"[кирилл, бог, путь, неисповедимый]"
...,...
50779,"[с, фамилия, 🙄]"
50780,"[наш, день, повезти, мужик, фамилия, но, суть,..."
50781,"[чудо, чудной, диво, дивный]"
50782,"[пригреться, уснуть, свариться, лягушка, так, ..."


### Добавление тегов из universal_tagset. Для этого нужно переименовать _taggers/universal_tagset/ru-rnс.map_ в _taggers/universal_tagset/ru-rnc-new.map_

In [16]:
def tagger(doc):
    tagged_tokens = []
    try:
        tagged_doc = nltk.pos_tag(doc, tagset='universal', lang = 'rus')
        for p in tagged_doc:
            if emoji.is_emoji(p[0]):
                token = p[0] + '_SYM'
            else:
                token = p[0] + '_' + p[1]
            tagged_tokens.append(token)
    except KeyError:
        print(doc)
        print('Тут ошибка формата!!')
    return(tagged_tokens)

In [17]:
comments_tagged = pd.DataFrame()

In [19]:
comments_tagged['tokens'] = comments['comment'].apply(tagger)

['ут', 'чк', 'м', 'ик', 'н', 'ки', 'тны', 'далее', 'м', 'нт', 'так', 'ы', 'ют', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'так', 'л', 'ь', 'зный', 'далее', 'з', 'так', 'ль', 'тв', 'м', 'так', 'г', 'чт', 'к', 'мль', 'н', 'дит', 'п', 'далее', 'лп', 'м', 'п', 'нт', 'г', 'н', 'чт', 'ия', 'так', 'л', 'язвить', 'й', 'вн', 'шни', 'г', 'з', 'эт', 'зн', 'ч', 'так', 'чт', 'н', 'ци', 'н', 'льн', 'б', 'з', 'п', 'н', 'ть', 'ия', 'н', 'дит', 'п', 'далее', 'г', 'з', 'й', 'ия', 'ки', 'вл', 'ти', 'далее', 'лжный', 'п', 'инять', 'чны', 'м', 'ы', 'з', 'щит', 'так', 'ны']
Тут ошибка формата!!
['ут', 'чк', 'тны', 'далее', 'м', 'нт', 'ющи', 'ук', 'ины', 'ближн', 'г', 'в', 'так', 'кит', 'вид', 'так', 'ль', 'тв', 'так', 'так', 'м', 'чт', 'ия', 'н', 'дит', 'п', 'далее', 'п', 'так', 'льный', 'нт', 'л', 'м', 'м', 'ик', 'н', 'й', 'зв', 'дки', 'далее', 'м', 'нты', 'п', 'нт', 'г', 'н', 'г', 'ят', 'н', 'м', 'так', 'м', 'чт', 'широта', 'зн', 'ют', 'п', 'ия', 'ч', 'нь', 'мн', 'г', 'эт', 'явля', 'так', 'г', 'з', 'й

In [20]:
comments_tagged

,tokens
0,[😭_SYM]
1,"[хранить_VERB, господь_NOUN, неравнодушный_ADJ..."
2,"[евгений_ADJ, еслиб_NOUN, бог_NOUN, такой_PRON..."
3,"[кирилл_VERB, полностью_ADV, согласный_VERB, 😢..."
4,"[кирилл_VERB, бог_NOUN, путь_NOUN, неисповедим..."
...,...
50779,"[с_ADP, фамилия_NOUN, 🙄_SYM]"
50780,"[наш_PRON, день_NOUN, повезти_VERB, мужик_NOUN..."
50781,"[чудо_NOUN, чудной_ADJ, диво_ADV, дивный_ADJ]"
50782,"[пригреться_VERB, уснуть_VERB, свариться_VERB,..."


## Создание тестового датафрейма для переобучения

In [54]:
test_df = comments_tagged[:2000]

In [57]:
for i in range(len(test_df)):
    flag = randint(1, 3)
    if flag == 1:
        test_df['tokens'].iloc[i] = ['федерация_NOUN', '👍_SYM', 'правда_NOUN', '🇷🇺_SYM', 'россия_NOUN', 'российский_ADJ', 'русский_ADJ']
    elif flag == 2:
        test_df['tokens'].iloc[i] = ['🐽_SYM', 'украинский_ADJ', '🐷_SYM', '🇺🇦_SYM', 'украина_NOUN', 'хохол_NOUN', 'хохлятский_ADJ']
    elif flag == 3:
        test_df['tokens'].iloc[i] = ['правильно_ADV', '👍_SYM', 'молодец_NOUN', 'правда_NOUN']     

In [60]:
test_data = test_df['tokens'].values.tolist()

In [62]:
test_data[:3]

[['🐽_SYM',
  'украинский_ADJ',
  '🐷_SYM',
  '🇺🇦_SYM',
  'украина_NOUN',
  'хохол_NOUN',
  'хохлятский_ADJ'],
 ['федерация_NOUN',
  '👍_SYM',
  'правда_NOUN',
  '🇷🇺_SYM',
  'россия_NOUN',
  'российский_ADJ',
  'русский_ADJ'],
 ['🐽_SYM',
  'украинский_ADJ',
  '🐷_SYM',
  '🇺🇦_SYM',
  'украина_NOUN',
  'хохол_NOUN',
  'хохлятский_ADJ']]

### Формирование списка списков

In [21]:
vk_data = comments_tagged['tokens'].values.tolist()

In [61]:
vk_data[:3]

[['😭_SYM'],
 ['хранить_VERB',
  'господь_NOUN',
  'неравнодушный_ADJ',
  'человек_NOUN',
  '🙏_SYM',
  '🕊_SYM'],
 ['евгений_ADJ', 'еслиб_NOUN', 'бог_NOUN', 'такой_PRON', 'допускать_VERB']]

# Модель Word2Vec

## Инициализация готовой модели *(RusVectōrēs news_upos_skipgram_300_5_2019)*

### Загрузка датасета лента.ру для создания словаря

In [ ]:
lenta_read = pd.read_csv('lenta_df_lern.csv')

In [ ]:
lenta_read.head(3)

In [ ]:
lenta_df = pd.DataFrame()

In [ ]:
lenta_read = lenta_read[:100000]

In [ ]:
lenta_df['tokens'] = lenta_read['text'].apply(str_to_list)

In [ ]:
lenta_df['tokens'] = lenta_df['tokens'].apply(tagger)

In [ ]:
lenta_df.head(3)

In [ ]:
lenta_data = lenta_df['tokens'].values.tolist()

### Инициализация модели и создание словаря на датасете лента.ру

In [38]:
model_vk_emoji = wv(vector_size=300, min_count=3, window=10)

In [39]:
model_vk_emoji.build_vocab(lenta_data)

In [40]:
len(model_vk_emoji.wv)

14252

In [41]:
model_vk_emoji.wv.vectors_lockf = np.ones(len(model_vk_emoji.wv))

In [42]:
model_vk_emoji.wv.intersect_word2vec_format('model.bin', lockf=1.0, binary=True)

In [43]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('беларусь_VERB', 0.457935631275177),
 ('европа_NOUN', 0.43242180347442627),
 ('русь_VERB', 0.42832908034324646),
 ('спб_NOUN', 0.4003465175628662),
 ('нтв_NOUN', 0.3854657709598541),
 ('будущее_VERB', 0.38034573197364807),
 ('российский_ADJ', 0.3762246072292328),
 ('москва_NOUN', 0.36797377467155457),
 ('путинский_ADJ', 0.35220542550086975),
 ('сергей_NOUN', 0.3440842628479004)]

### Дополнение словаря на тестовых данных

In [25]:
model_vk_emoji.build_vocab(test_data)  # , update=True

In [44]:
len(model_vk_emoji.wv)

14252

In [45]:
model_vk_emoji.wv.similarity('россия_NOUN', '🇷🇺_SYM')

0.046223182

In [46]:
model_vk_emoji.wv.most_similar('россия_NOUN')

[('вашингтон_NOUN', 0.24730491638183594),
 ('обь_NOUN', 0.22349801659584045),
 ('поправляться_VERB', 0.2047533541917801),
 ('ло_NOUN', 0.19991838932037354),
 ('💂_SYM', 0.19526289403438568),
 ('шаман_NOUN', 0.18780206143856049),
 ('пригожин_VERB', 0.18474437296390533),
 ('дугин_NOUN', 0.18116363883018494),
 ('зоя_NOUN', 0.18098628520965576),
 ('котрый_ADJ', 0.17992910742759705)]

## Обучение модели на текстах ВК

In [47]:
model_vk_emoji.train(test_data, total_examples=model_vk_emoji.corpus_count, epochs=1)

(345824, 393800)

In [49]:
model_vk_emoji.wv.similarity('россия_NOUN', '🇷🇺_SYM')

0.06487592

In [50]:
model_vk_emoji.wv.most_similar('🇷🇺_SYM')

[('вмера_NOUN', 0.18435297906398773),
 ('ночью_NOUN', 0.1774526834487915),
 ('илсоя_VERB', 0.17638114094734192),
 ('свински_ADV', 0.17635108530521393),
 ('саня_VERB', 0.17632462084293365),
 ('православный_VERB', 0.17594033479690552),
 ('уха_NOUN', 0.17292630672454834),
 ('вежливость_NOUN', 0.17083483934402466),
 ('водка_NOUN', 0.1684546023607254),
 ('нех_NOUN', 0.16834816336631775)]

# Подготовка датасета для анализа тональности  *(дополнительно)* 

In [20]:
negative = ['😡', '👿', '😾', '😤', '😠', '🤬', '💀', '☠️']

In [21]:
sarcastic = ['💩', '🤡', '🙃', '🤢', '😈']  # Нужно дополнить

In [22]:
positive = ['😀', '😃', '😉']  # Нужно дополнить